In [1]:
import os
import glob
import numpy as np
import pandas as pd
from functions import split_preds, metrics, measure_csv_df
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
os.chdir("/Users/pestephan/Documents/GitHub/Machine-Learning-using-Sports-Data/Predicting-PremLeague-Outcomes-With-ML/files/Datasets")

Data Taken from http://www.football-data.co.uk/englandm.php

In [2]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [3]:
#combine all files in the list
master_sheet = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
master_sheet.to_csv( "master_sheet.csv", index=False, encoding='utf-8-sig')

Reduce table to using columns needed for the model

Columns Required:

Date, Time HomeTeam, AwayTeam, FullTimeHG, FullTimeAG, FullTimeRresults, HalfTimeHG, HalfTimeAG, HomeShots, AwayShots, HomeShotsTarger, AwayShotsTarger, HomeCorners, AwayCorners, HomeFouls, AwayFouls, HomeRed, AwayRed

Columns To Consider:

Attendance, Referee

In [4]:
subset_master = master_sheet[["Date", "Time", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HTHG", "HTAG", "HS", "AS", "HST", "AST", "HC", "AC", "HF", "AF", "HR", "AR"]]
subset_master = subset_master.fillna(0)
# 0 = Home Wins, 1 = Draw, 2 = Away Wins
subset_master.FTR.replace(['H', 'D', 'A'], [0, 1, 2], inplace = True)

In [5]:
subset_master.head()

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HS,AS,HST,AST,HC,AC,HF,AF,HR,AR
0,13/08/11,0,Blackburn,Wolves,1.0,2.0,2,1.0,1.0,16.0,13.0,8.0,4.0,12.0,6.0,14.0,10.0,0.0,0.0
1,13/08/11,0,Fulham,Aston Villa,0.0,0.0,1,0.0,0.0,13.0,7.0,9.0,1.0,2.0,3.0,10.0,18.0,0.0,0.0
2,13/08/11,0,Liverpool,Sunderland,1.0,1.0,1,1.0,0.0,11.0,15.0,4.0,6.0,6.0,3.0,17.0,12.0,0.0,0.0
3,13/08/11,0,Newcastle,Arsenal,0.0,0.0,1,0.0,0.0,6.0,9.0,1.0,4.0,2.0,5.0,9.0,11.0,0.0,1.0
4,13/08/11,0,QPR,Bolton,0.0,4.0,2,0.0,1.0,13.0,13.0,7.0,7.0,3.0,2.0,9.0,16.0,1.0,0.0


In [6]:
msk = np.random.rand(len(subset_master)) < 0.75

train_df = subset_master[msk]
test_df = subset_master[~msk]

X_train = train_df.drop(columns = ['Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', "HTHG", "HTAG"])
y_train = train_df[['FTR']] 
X_test = test_df.drop(columns = ['Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', "HTHG", "HTAG", ])
y_test = test_df[['FTR']]

In [7]:
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=10000, multi_class='ovr', verbose=0)

clf.fit(X_train, np.ravel(y_train.values))

y_pred = clf.predict_proba(X_test)
y_pred = pd.DataFrame(y_pred)
split_preds(subset_master, y_pred)

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HS,...,AST,HC,AC,HF,AF,HR,AR,HomeWinPerc,DrawPerc,AwayPerc
0,13/08/11,0,Blackburn,Wolves,1.0,2.0,2,1.0,1.0,16.0,...,4.0,12.0,6.0,14.0,10.0,0.0,0.0,0.330279,0.359835,0.309886
1,13/08/11,0,Fulham,Aston Villa,0.0,0.0,1,0.0,0.0,13.0,...,1.0,2.0,3.0,10.0,18.0,0.0,0.0,0.173488,0.234722,0.591790
2,13/08/11,0,Liverpool,Sunderland,1.0,1.0,1,1.0,0.0,11.0,...,6.0,6.0,3.0,17.0,12.0,0.0,0.0,0.045142,0.274527,0.680331
3,13/08/11,0,Newcastle,Arsenal,0.0,0.0,1,0.0,0.0,6.0,...,4.0,2.0,5.0,9.0,11.0,0.0,1.0,0.582069,0.252071,0.165861
4,13/08/11,0,QPR,Bolton,0.0,4.0,2,0.0,1.0,13.0,...,7.0,3.0,2.0,9.0,16.0,1.0,0.0,0.033919,0.220514,0.745567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,24/05/15,0,Leicester,QPR,5.0,1.0,0,2.0,0.0,22.0,...,2.0,5.0,6.0,7.0,6.0,0.0,0.0,0.754266,0.203683,0.042051
377,24/05/15,0,Man City,Southampton,2.0,0.0,0,1.0,0.0,15.0,...,4.0,8.0,4.0,13.0,8.0,0.0,0.0,0.234773,0.316662,0.448566
378,24/05/15,0,Newcastle,West Ham,2.0,0.0,0,0.0,0.0,17.0,...,1.0,2.0,3.0,9.0,9.0,0.0,0.0,0.587261,0.313202,0.099538
379,24/05/15,0,Stoke,Liverpool,6.0,1.0,0,5.0,0.0,15.0,...,4.0,3.0,9.0,13.0,4.0,0.0,0.0,0.427112,0.303218,0.269669


In [8]:
metrics(subset_master)

False    56.844574
True     43.155426
Name: Comparison, dtype: float64
0    45.329270
2    30.113404
1    24.557325
Name: FTR, dtype: float64
0    87.418193
2     9.884920
1     2.696887
Name: PredResult, dtype: float64


In [9]:
subset_master

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HS,...,AC,HF,AF,HR,AR,HomeWinPerc,DrawPerc,AwayPerc,PredResult,Comparison
0,13/08/11,0,Blackburn,Wolves,1.0,2.0,2,1.0,1.0,16.0,...,6.0,14.0,10.0,0.0,0.0,0.330279,0.359835,0.309886,1,False
1,13/08/11,0,Fulham,Aston Villa,0.0,0.0,1,0.0,0.0,13.0,...,3.0,10.0,18.0,0.0,0.0,0.173488,0.234722,0.591790,2,False
2,13/08/11,0,Liverpool,Sunderland,1.0,1.0,1,1.0,0.0,11.0,...,3.0,17.0,12.0,0.0,0.0,0.045142,0.274527,0.680331,2,False
3,13/08/11,0,Newcastle,Arsenal,0.0,0.0,1,0.0,0.0,6.0,...,5.0,9.0,11.0,0.0,1.0,0.582069,0.252071,0.165861,0,False
4,13/08/11,0,QPR,Bolton,0.0,4.0,2,0.0,1.0,13.0,...,2.0,9.0,16.0,1.0,0.0,0.033919,0.220514,0.745567,2,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,24/05/15,0,Leicester,QPR,5.0,1.0,0,2.0,0.0,22.0,...,6.0,7.0,6.0,0.0,0.0,0.754266,0.203683,0.042051,0,True
377,24/05/15,0,Man City,Southampton,2.0,0.0,0,1.0,0.0,15.0,...,4.0,13.0,8.0,0.0,0.0,0.234773,0.316662,0.448566,2,False
378,24/05/15,0,Newcastle,West Ham,2.0,0.0,0,0.0,0.0,17.0,...,3.0,9.0,9.0,0.0,0.0,0.587261,0.313202,0.099538,0,True
379,24/05/15,0,Stoke,Liverpool,6.0,1.0,0,5.0,0.0,15.0,...,9.0,13.0,4.0,0.0,0.0,0.427112,0.303218,0.269669,0,True


In [10]:
current_season = pd.read_csv("/Users/pestephan/Documents/GitHub/Machine-Learning-using-Sports-Data/Predicting-PremLeague-Outcomes-With-ML/2020-2021.csv")
current_season_subset = current_season[["Date", "Time", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HTHG", "HTAG", "HS", "AS", "HST", "AST", "HC", "AC", "HF", "AF", "HR", "AR"]]
current_season_subset_pred = current_season_subset.drop(columns = ['Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', "HTHG", "HTAG"])
current_season_subset_pred.head()

,HS,AS,HST,AST,HC,AC,HF,AF,HR,AR
0,5,13,2,6,2,3,12,12,0,0
1,5,9,3,5,7,3,14,11,0,0
2,22,6,6,3,9,0,9,6,0,0
3,15,15,3,2,8,7,13,7,0,0
4,7,13,1,7,2,5,12,9,0,0


In [11]:
y_pred = clf.predict_proba(current_season_subset_pred)
y_pred = pd.DataFrame(y_pred)

In [12]:
current_season_subset = measure_csv_df(current_season_subset,clf)
current_season_subset.FTR.replace(['H', 'D', 'A'], [0, 1, 2], inplace = True)
current_season_subset

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HS,...,AST,HC,AC,HF,AF,HR,AR,HomeWinPerc,DrawPerc,AwayPerc
0,12/09/2020,12:30,Fulham,Arsenal,0,3,2,0,1,5,...,6,2,3,12,12,0,0,0.141943,0.320171,0.537886
1,12/09/2020,15:00,Crystal Palace,Southampton,1,0,0,1,0,5,...,5,7,3,14,11,0,0,0.161147,0.373707,0.465146
2,12/09/2020,17:30,Liverpool,Leeds,4,3,0,3,2,22,...,3,9,0,9,6,0,0,0.329883,0.303962,0.366155
3,12/09/2020,20:00,West Ham,Newcastle,0,2,2,0,0,15,...,2,8,7,13,7,0,0,0.386597,0.360112,0.253291
4,13/09/2020,14:00,West Brom,Leicester,0,3,2,0,0,7,...,7,2,5,12,9,0,0,0.091506,0.296632,0.611863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,07/02/2021,12:00,Tottenham,West Brom,2,0,0,0,0,13,...,1,8,0,16,12,0,0,0.486120,0.361332,0.152548
221,07/02/2021,14:00,Wolves,Leicester,0,0,1,0,0,13,...,3,8,2,11,15,0,0,0.114076,0.397291,0.488633
222,07/02/2021,16:30,Liverpool,Man City,1,4,2,0,0,8,...,5,6,1,13,8,0,0,0.144973,0.328041,0.526986
223,07/02/2021,19:15,Sheffield United,Chelsea,1,2,2,0,1,8,...,3,1,8,8,8,0,0,0.510663,0.289734,0.199603


In [13]:
metrics(current_season_subset)

True     60.444444
False    39.555556
Name: Comparison, dtype: float64
2    40.000000
0    36.888889
1    23.111111
Name: FTR, dtype: float64
0    48.888889
2    39.555556
1    11.555556
Name: PredResult, dtype: float64
